In [1]:
from bs4 import BeautifulSoup
from pathlib import Path
from PIL import Image # Importando o módulo Pillow para abrir a imagem no script
from PyPDF2 import PdfFileReader
from pdf2jpg import pdf2jpg
from tabula import read_pdf
import pytesseract # Módulo para a utilização da tecnologia OCR
import requests
from PIL import Image 
import enade as ed
import re
import os

### Image to String

In [ ]:
# IMAGEM = 'paginaSimples2.jpg'
IMAGEM = '22.jpg'
# IMAGEM = 'paginaDupla2.jpg'
# IMAGEM = 'paginaDuplaQ2.jpg'

In [2]:
def JPG_to_TXT(IMAGEM):
    texto = pytesseract.image_to_string(Image.open(IMAGEM))
#     print(texto)
    return texto

In [3]:
def analisarIMG(IMAGEM):
    texto = pytesseract.image_to_string(Image.open(IMAGEM))
    disc = ['Discursivas', 'Discursiva', 'DISCURSIVAS', 'DISCURSIVA', 'Questionario', 'QUESTIONARIO', 'Questionário', 'QUESTIONÁRIO']
    for wd in disc:
        questao = texto.find(wd)
        if questao != -1: return -1
    
    return 0

In [ ]:
texto = pytesseract.image_to_string(Image.open(IMAGEM))
print(texto)

### Página Dupla ou Simples?

In [4]:
def getQuestao(IMAGEM):
    
    texto = JPG_to_TXT(IMAGEM)
    
    numQuestoes = 0

    questao = 0
    while questao != -1:
        
        questao = texto.find('QUESTAO')
        
        if questao != -1:
            numQuestoes+=1
            texto = texto.replace('QUESTAO', 'OK', 1)
            
        
        if questao == -1:
            
            questao = texto.find('Questao')

            if questao != -1:
                numQuestoes+=1
                texto = texto.replace('Questao', 'OK', 1)
                

    questao = texto.find('Area livre')
    if questao != -1:
        return numQuestoes, 'AL'
    
    return numQuestoes, '-1'

In [5]:
def pgSD(IMAGEM):
    
    im = Image.open(r""+IMAGEM)
    
    width, height = im.size
    print(width, height)
    
    #Left, top, rigth, bottom
    imR = im.crop((0, 300, width/2, height))
    imR.save('img1.jpg')
    
    imR = im.crop((width/2, 300, width, height))
    imR.save('img2.jpg')
    
    img1Q, lx = getQuestao('img1.jpg')
    img2Q, lx = getQuestao('img2.jpg')
    
    if img1Q == 0 and img2Q == 0:
        return 'none', -1, -1
    
    elif img1Q > 0 and img2Q > 0:
        return 'pgD', img1Q, img2Q
    
    elif img1Q > 0 and img2Q == 0:
        return 'pgS', img1Q, img2Q
    
    elif img1Q == 0 and img2Q > 0:
        input("FATAL ERROR")
        
    return 'none', -1, -1

### Crop Image

In [6]:
# pagina = pgSD(IMAGEM)

In [7]:
def cropImage(IMAGEM, ID, diretorio):
    
    print('\n\nProcessando pagina: ', IMAGEM)
    
    if analisarIMG(IMAGEM) == -1:
        print("Questao Discursiva...\nProx...")
        return ID, -1
    
    im = Image.open(r""+IMAGEM)
    
    width, height = im.size

    pagina, img1Q, img2Q = pgSD(IMAGEM)
    if pagina == 'none': 
        print('Nenhuma questão encontrada na pagina: ', IMAGEM)
        return ID, -1
    
    print('Pagina: ', pagina, img1Q, img2Q)
     #Left, top, rigth, bottom
    coordenadasIMG = []
    
    if pagina == 'pgS':
        coordenadasIMG.append((0, 300, width, height-250))
        
    elif pagina == 'pgD':
        coordenadasIMG.append((0, 300, width/2, height-250))
        coordenadasIMG.append((width/2, 300, width, height-250))
        
    elif True:
        input("ERRO AO IDENTIFICAR QUESTOES")
        return ID, -1
    
    getQ = 0
    for i in range(len(coordenadasIMG)):
    
        left, top, right, bottom = coordenadasIMG[i][0], coordenadasIMG[i][1], coordenadasIMG[i][2], coordenadasIMG[i][3]
        print(left, top, right, bottom)
        
        saveTOP = saveBT = cont = 0
        if pagina == 'pgS': 
            result, AL = getQuestao(IMAGEM)
            if AL != 'AL':
                saveBT = bottom+50
        x = 400
        while x < height-200:

            imR = im.crop((left, top, right, x))
            imR.save('img.jpg')

            if saveTOP == 0:
                result, AL = getQuestao('img.jpg')
                if result != 0:
                    saveTOP = x
                    x = x + 100
                    top = top + 100

            elif saveBT == 0:
                result, AL = getQuestao('img.jpg')
                if AL == 'AL': saveBT = x
                if result != 0: saveBT = x

            if saveTOP != 0 and saveBT != 0:
                imR = im.crop((left, saveTOP+10, right, saveBT-50))
                imR.save(diretorio+'/'+str(ID)+'.jpg')
                ID+=1
                saveTOP = saveBT = 0
                print("1) Get Question "+str(getQ)+" Success")
                getQ+=1
                if pagina == 'pgS': break
                
            x+=10
            top+=10
            
        if saveTOP != 0 and saveBT == 0:
            imR = im.crop((left, saveTOP-5, right, bottom))
            imR.save(diretorio+'/'+str(ID)+'.jpg')
            ID+=1
            saveTOP = saveBT = 0
            print("2) Get Question "+str(getQ)+" Success")
            getQ+=1


    print('Numero de questoes capturadas: ', getQ)
    print('Total de questoes: ', img1Q+img2Q)
    return ID, 0

In [14]:
ID = 0

for ano in dicEnade.keys():
    
    print('>> Ano: ', ano)
    
    for area in dicEnade[ano]:
        
        print('   >> Area: ', area)
        os.mkdir('EnadeProvas/'+ano+'/'+area+'/Questoes')
        for i in range(1, 40):
            ID, OK = cropImage('EnadeProvas/'+ano+'/'+area+'/Prova.pdf_dir/'+str(i)+'_Prova.pdf.jpg', ID, 'EnadeProvas/'+ano+'/'+area+'/Questoes')

            if OK == -1:
                print('Pagina Ignorada: ', i)
            elif True: print('Pagina processada com Sucesso!: ', i)
               

>> Ano:  2017
   >> Area:  Arquitetura e Urbanismo


Processando pagina:  EnadeProvas/2017/Arquitetura e Urbanismo/Prova.pdf_dir/1_Prova.pdf.jpg
Questao Discursiva...
Prox...
Pagina Ignorada:  1


Processando pagina:  EnadeProvas/2017/Arquitetura e Urbanismo/Prova.pdf_dir/2_Prova.pdf.jpg
2617 3444
Nenhuma questão encontrada na pagina:  EnadeProvas/2017/Arquitetura e Urbanismo/Prova.pdf_dir/2_Prova.pdf.jpg
Pagina Ignorada:  2


Processando pagina:  EnadeProvas/2017/Arquitetura e Urbanismo/Prova.pdf_dir/3_Prova.pdf.jpg
Questao Discursiva...
Prox...
Pagina Ignorada:  3


Processando pagina:  EnadeProvas/2017/Arquitetura e Urbanismo/Prova.pdf_dir/4_Prova.pdf.jpg
2617 3444
Pagina:  pgS 1 0
0 300 2617 3194
1) Get Question 0 Success
Numero de questoes capturadas:  1
Total de questoes:  1
Pagina processada com Sucesso!:  4


Processando pagina:  EnadeProvas/2017/Arquitetura e Urbanismo/Prova.pdf_dir/5_Prova.pdf.jpg
2617 3444
Pagina:  pgS 1 0
0 300 2617 3194
1) Get Question 0 Success
Numero de 

FileNotFoundError: [Errno 2] No such file or directory: 'EnadeProvas/2017/Arquitetura e Urbanismo/Prova.pdf_dir/23_Prova.pdf.jpg'

### read dicEnade

In [8]:
def getDicEnade():
    
    arqDic = open('dicEnade.txt')
    
    dicEnade = {}
    
    for line in arqDic:
        
        values = line.split('::')
        
        ano, area = values[0], values[1]
        area = area.replace('\n', '', 1)
        
        if ano not in dicEnade.keys():
            dicEnade[ano] = []
            
        dicEnade[ano].append(area)
    
    return dicEnade

### PDF to JPG

In [9]:
def convert_PDF_to_JPG(namePDF, ano, area):

    inputpath = 'EnadeProvas/'+ano+'/'+area+'/'+namePDF
    outputpath = 'EnadeProvas/'+ano+'/'+area+'/'
    result = pdf2jpg.convert_pdf2jpg(inputpath, outputpath, pages="ALL")
    # print(result)

In [12]:
dicEnade = getDicEnade()
# print(dicEnade)

# for ano in dicEnade.keys():
#     print('>> Ano: ', ano)
#     for area in dicEnade[ano]:
#         print('   >> Area: ', area)
#         convert_PDF_to_JPG('Prova.pdf', ano, area)
       
# print('FIM')

### Get Questions